In [ ]:
import gym
from gym.spaces import Discrete, Box

import numpy as np

In [ ]:
def calc_reward(items, state, max_weight, burst_reward):
    reward = 0
    weight = 0
    
    for i in range(len(state)):
        reward += items[i][0] * state[i]
        weight += items[i][1] * state[i]
    
    if weight > max_weight or min(state) < 0:
        reward = burst_reward
    
    return reward, weight

In [ ]:
def action_to_state(action, bit_num):
    acts = np.reshape(action, (-1, bit_num))
    
    b_acts = np.pad(acts, ((0, 0), (8 - bit_num, 0)))
    b_acts = np.vectorize(lambda x: 1 if x >= 0.5 else 0)(b_acts)
    
    return list(np.packbits(b_acts))

# Env

In [ ]:
class Knapsack(gym.Env):
    def __init__(self, config):
        self.items = config["items"]
        self.bit_num = min(config["bit_num"], 8)
        self.max_weight = config["max_weight"]
        self.burst_reward = config["burst_reward"]
        
        h = 2 ** self.bit_num - 1
        
        self.action_space = Box(0, 1, shape = (len(self.items) * self.bit_num, ))
        self.observation_space = Box(0, h, shape = (len(self.items), ))

    def reset(self):
        return [0 for _ in self.items]

    def step(self, action):
        state = action_to_state(action, self.bit_num)
        
        reward, _ = calc_reward(self.items, state, self.max_weight, self.burst_reward)
        
        return state, reward, True, {}

In [ ]:
items = [
    [120, 10],
    [130, 12],
    [80, 7],
    [100, 9],
    [250, 21],
    [185, 16]
]


In [ ]:
config = {
    "env": Knapsack, 
    "env_config": {"items": items, "bit_num": 3, "max_weight": 65, "burst_reward": -100}
}


In [ ]:
import ray

#ray.shutdown()
ray.init()

In [ ]:
from ray.rllib.agents.ppo import PPOTrainer

trainer = PPOTrainer(config = config)

In [ ]:
from ray.rllib.agents.ddpg import DDPGTrainer

trainer = DDPGTrainer(config = config)

# Train

In [ ]:
from ray.tune.logger import pretty_print

for _ in range(50):
    r = trainer.train()
    print(pretty_print(r))


# Evaluate

In [ ]:
import collections

rs = []

for _ in range(100):
    s = [0 for _ in range(len(items))]
    a = trainer.compute_action(s)
    
    s = action_to_state(a, config["env_config"]["bit_num"])
    
    r, w = calc_reward(items, s, config["env_config"]["max_weight"], -1)
    
    rs.append(r)
    
    print(f"{s}, {r}, {w}")

collections.Counter(rs)

# Save

In [ ]:
checkpoint = trainer.save()
checkpoint

# Load

In [ ]:
trainer.restore(checkpoint)
